In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
import nest_asyncio

nest_asyncio.apply()

# Set up your local LLM using Ollama.
Settings.llm = Ollama(
    model="qwq:32b",
    function_call_support=True,  # Enable function call support if available.
    # Additional parameters (e.g., endpoint) can be passed here if needed.
)

# For embeddings, we're using the HuggingFace model.
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [2]:
from typing import List
#-------------------------------------------------------------
# Tool 1 : Function that returns the list of items in an order
#-------------------------------------------------------------
def get_order_items(order_id: int) -> List[str]:
    """Given an order Id, this function returns the 
    list of items purchased for that order."""
    order_items = {
        1001: ["Laptop", "Mouse"],
        1002: ["Keyboard", "HDMI Cable"],
        1001: ["Laptop", "Keyboard"]
    }
    return order_items.get(order_id, [])


#-------------------------------------------------------------
# Tool 2 : Function that returns the delivery date for an order
#-------------------------------------------------------------
def get_delivery_date(order_id: int) -> str:
    """Given an order Id, this function returns the 
    delivery date for that order."""
    delivery_dates = {
        1001: "10-Jun",
        1002: "12-Jun",
        1003: "08-Jun"
    }
    return delivery_dates.get(order_id, "")


#----------------------------------------------------------------
# Tool 3 : Function that returns maximum return days for an item
#----------------------------------------------------------------
def get_item_return_days(item: str) -> int:
    """Given an Item, this function returns the return support
    for that order. The return support is in number of days."""
    item_returns = {
        "Laptop": 30,
        "Mouse": 15,
        "Keyboard": 15,
        "HDMI Cable": 5
    }
    return item_returns.get(item, 45)


#-------------------------------------------------------------
# Tool 4 : Vector DB that contains customer support contacts
#-------------------------------------------------------------
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool

# Load the customer service document.
support_docs = SimpleDirectoryReader(input_files=["Customer Service.pdf"]).load_data()

# Split the document into nodes/chunks.
splitter = SentenceSplitter(chunk_size=1024)
support_nodes = splitter.get_nodes_from_documents(support_docs)

# Build the vector index from the support nodes.
support_index = VectorStoreIndex(support_nodes)
support_query_engine = support_index.as_query_engine()

# Now you can use your tools and vector index alongside the local LLM.

In [3]:
from llama_index.core.tools import FunctionTool, QueryEngineTool

# Create function tools for your defined functions.
order_item_tool = FunctionTool.from_defaults(fn=get_order_items)
delivery_date_tool = FunctionTool.from_defaults(fn=get_delivery_date)
return_policy_tool = FunctionTool.from_defaults(fn=get_item_return_days)

# Create a query engine tool for the customer support vector index.
support_tool = QueryEngineTool.from_defaults(
    query_engine=support_query_engine,
    description="Customer support policies and contact information",
)

In [4]:
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner

# Setup the Agent worker with all the tools using the local LLM (Settings.llm)
agent_worker = FunctionCallingAgentWorker.from_tools(
    [order_item_tool, 
     delivery_date_tool,
     return_policy_tool,
     support_tool
    ],
    llm=Settings.llm, 
    verbose=True
)

# Create an Agent Orchestrator with LlamaIndex
agent = AgentRunner(agent_worker)

In [5]:
#Get return policy for an order
response = agent.query(
    "What is the return policy for order number 1001"
)

print("\n Final output : \n", response)

Added user message to memory: What is the return policy for order number 1001
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1001}
=== Function Output ===
['Laptop', 'Keyboard']
=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "Laptop"}
=== Function Output ===
30
=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "Laptop"}
=== Function Output ===
30
=== LLM Response ===
<think>
Okay, let me figure out how to handle this. The user asked about the return policy for order 1001.

First, I called get_order_items(1001) and got ['Laptop', 'Keyboard']. 

Then I needed the return days for each item. I called get_item_return_days for "Laptop" twice by mistake. Wait, that's an error. The second call should have been for "Keyboard". Oh no, I made a mistake there.

The responses were 30 and 30, but since the second item was supposed to be Keyboard, maybe its return period is different? But with m

In [6]:
# Three part question
response = agent.query(
    "When is the delivery date and items shipped for order 1003 and how can I contact customer support?"
)

print("\n Final output : \n", response)

Added user message to memory: When is the delivery date and items shipped for order 1003 and how can I contact customer support?
=== Calling Function ===
Calling function: get_delivery_date with args: {"order_id": 1003}
=== Function Output ===
08-Jun
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1003}
=== Function Output ===
[]
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "Customer support contact information"}
=== Function Output ===
<think>
Okay, I need to figure out the customer support contact information based on the provided context. Let me look back at the context given.

The context says that customer service is available Monday to Friday from 8 AM to 5 PM. The phone number listed is 1-987-654-3210, and the email is support@company.com. The user is asking for the contact information, so I should include both the phone number and the email address. 

I have to make sure not to mention the context directly.

In [7]:
#Question about an invalid order number
response = agent.query(
    "What is the return policy for order number 1004"
)

print("\n Final output : \n", response)

Added user message to memory: What is the return policy for order number 1004
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===
[]
=== LLM Response ===
<think>
Okay, let me try to figure this out. The user is asking about the return policy for order number 1004. First, I need to get the items in that order. I called the get_order_items function with order_id 1004, but the response was an empty list. Hmm, that's odd. Maybe there was an error or the order doesn't exist? But I should check if there's another way to proceed.

Wait, perhaps the order ID is incorrect, or the function didn't retrieve any data. Since there are no items listed, I can't use get_item_return_days for specific items. Alternatively, maybe the user needs general return policy info. The query_engine_tool can access customer support policies. Let me use that to provide a helpful answer even without the order details.

I'll explain that the system couldn't f